In [1]:
# Library imports
import pandas as pd
import networkx as nx
import numpy as np
from NetworkGraph import NetworkGraph
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df=pd.read_csv("Output\\emails_all_bck.csv")

df.shape

(2811799, 2)

In [3]:
df2 = df.groupby(['From','To']).size().reset_index()

df2.columns = ['From', 'To', 'Weight']

df2.head()

,From,To,Weight
0,'todd'.delahoussaye@enron.com,'todd'.delahoussaye@enron.com,5
1,'todd'.delahoussaye@enron.com,ajay.sharma@enron.com,5
2,'todd'.delahoussaye@enron.com,anne.bike@enron.com,1
3,'todd'.delahoussaye@enron.com,bianca.ornelas@enron.com,5
4,'todd'.delahoussaye@enron.com,brant.reves@enron.com,5


The NetworkGraph class takes in a list of tuples containing the edges - as such we need to convert the grouped data frame before generating the graph object

In [4]:
# Store the smaller dataset for later use
# df2.to_csv('Output/grouped_mails.csv')

In [5]:
edges = [(x[0], x[1], x[2]) for x in list(df2.values)]
len(edges)

288695

In [6]:
G = NetworkGraph(edges)

G.graph.number_of_nodes()

71971

In [7]:
G.graph.number_of_edges()

288695

In [8]:
nx.number_connected_components(G.graph.to_undirected())

1124

This is a surprisingly high number of connected components and is most likely down to the witheld data - some of these gaps should be filled in with the remaining data so that the number of components decreases. Regardless, the focus should be on individual components - we can focus on those with features / individuals of interest.
Lets first see how many nodes/edges each component has and then identify cycles / cliques withtin them

In [9]:
subgraphs = nx.weakly_connected_component_subgraphs(G.graph, copy=True)

In [10]:
components = []

for g in subgraphs:
    params = (g.number_of_edges(), g.number_of_nodes())
    components.append(params)
    
components.sort(reverse=True)
components[:10]

[(287014, 69476),
 (100, 101),
 (42, 43),
 (28, 29),
 (22, 23),
 (21, 14),
 (14, 15),
 (13, 14),
 (12, 13),
 (10, 11)]

It looks like most of the graph is contained in a single large component with the remaining pieces small components with just a few nodes. Lets try and find any cycles in any of the smaller ones

In [11]:
subgraphs = nx.weakly_connected_component_subgraphs(G.graph, copy=True)

for g in subgraphs:
    if g.number_of_edges() < 2000:
        net = NetworkGraph(g)
        net.findCycles()
        if len(net.cycles) > 0:
            net.printCycles()
    else:
        # Store the maximal component for later
        G2 = NetworkGraph(g)

merlyn@stonehenge.com->cp@onsitetech.com->tex@off.org->merlyn@stonehenge.com
merlyn@stonehenge.com->cp@onsitetech.com->merlyn@stonehenge.com
merlyn@stonehenge.com->cp@onsitetech.com->tex@off.org->merlyn@stonehenge.com
merlyn@stonehenge.com->cp@onsitetech.com->merlyn@stonehenge.com


There doesn't look to be anything of interest in the small components so we can probably discard them when doing the analysis

As it stands this graph is too big to perform satisfactory analysis on. However, based on the problem statement we know of 5 users who have already been convicted - Chief Executive Officer Jeff Skilling, CEO and chairman Ken Lay, Chief Financial Officer Andrew Fastow, Chief Accounting Officer Rick Causey and Corporate Treasurer Ben Glisan. We want to take these known users as the starting point and only look at a reduced subset featuring users in close proximity to these. Proximity will be computed based on path distance within the graph.

We'll compute the distance between these 5 and all other users then, for each of the 5 take the 100 closest to each of them and drill down to the subgraph formed by these users.

The list of nodes that we want to focus on is therefore:
- 'jeff.skilling@enron.com'
- 'kenneth.lay@enron.com'
- 'andrew.fastow@enron.com'
- 'ben.glisan@enron.com'
- 'richard.causey@enron.com'


In [12]:
# Find those present in the graph

convicts = ['jeff.skilling@enron.com', 'kenneth.lay@enron.com',
           'andrew.fastow@enron.com', 'ben.glisan@enron.com',
           'richard.causey@enron.com']

present_convicts = [x for x in convicts if x in list(G.graph.nodes())]
present_convicts

['jeff.skilling@enron.com',
 'kenneth.lay@enron.com',
 'andrew.fastow@enron.com',
 'ben.glisan@enron.com',
 'richard.causey@enron.com']

In [13]:
# Store prximities for each convict
distances = {}
for c in convicts:
    distances[c] = nx.nx.shortest_path_length(G.graph, source=c, target=None, weight='weight')
    print len(distances[c])

47375
47375
47375
47375
47375


All 5 convicts have 47,375 nodes reachable, which represents a significant reduction in the graph size. Let's first focus on just this sub-graph and see if the methods for finding cycles, blackholes and cliques can be run

In [14]:
subgraph_nodes = []

for c in convicts:
    subgraph_nodes += distances[c].keys()

subgraph_nodes = list(set(subgraph_nodes))
len(subgraph_nodes)

47375

In [15]:
g2 = G.graph.subgraph(subgraph_nodes)

In [16]:
# Create a network graph using this subset
G2 = NetworkGraph(g2)

Using the implemented algorithms, identify the cycles, cliques and blackholes within the graph

In [17]:
#G.slowCycles()

#len(G.cycles)

# Runtime is too slow - need to look at optimising the algorithm
# After ~ 1.5hrs 700 cycles were found

In [18]:
# len(G.cycles)

In [19]:
# lengths = [len(c) for c in G.cycles]
# lengths.sort(reverse=True)

# lengths[:20]

In [20]:
#G.findMaximalCliques()
#len(G.maximal_cliques)

# Maximum recursion depth error with this method

In [24]:
G.findBlackHoles(2)

len(G.black_holes)

# Despite the pruning, the brute-force part of the algorithm is still too intensive
# Memory error on the step to calculate all possible subsets of the remaining nodes

4168